In [14]:
import numpy as np

data = np.loadtxt('pytorch_exercise/AI_Phys_Hw_2/train_data.txt', delimiter=' ')  # 假设空格分隔
x1 = data[:, 0]  # 第一列
x2 = data[:, 1]  # 第二列
U = data[:, 2]   # 第三列

In [15]:
x1_min, x1_max = -1.5, 1.0
x2_min, x2_max = -0.5, 2.0

x1_normalized = (x1 - x1_min) / (x1_max - x1_min)
x2_normalized = (x2 - x2_min) / (x2_max - x2_min)
X = np.stack([x1_normalized, x2_normalized], axis=1)  # 形状 [500, 2]
y = U  # 形状 [500]

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [18]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

model = MLP()

In [19]:
criterion = nn.L1Loss()  # L1 Loss 对应 MAE

In [20]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=1000, patience=50):
    best_val_mae = float('inf')
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(num_epochs):
        # 训练阶段
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)
        train_loss /= len(train_loader.dataset)
        
        # 验证阶段
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                val_loss += criterion(outputs, y_batch).item() * X_batch.size(0)
        val_loss /= len(val_loader.dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train MAE: {train_loss:.4f}, Val MAE: {val_loss:.4f}')
        
        # 早停
        if val_loss < best_val_mae:
            best_val_mae = val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break
    
    # 加载最佳模型
    model.load_state_dict(best_model_state)
    return model, best_val_mae

# 训练模型
model, train_U_mae = train_model(model, train_loader, val_loader, criterion, optimizer)
print(f"训练集最终 MAE: {train_U_mae:.4f}")

Epoch 1/1000, Train MAE: 0.3521, Val MAE: 0.4570
Epoch 2/1000, Train MAE: 0.3604, Val MAE: 0.4682
Epoch 3/1000, Train MAE: 0.3594, Val MAE: 0.4553
Epoch 4/1000, Train MAE: 0.3437, Val MAE: 0.4660
Epoch 5/1000, Train MAE: 0.3550, Val MAE: 0.4665
Epoch 6/1000, Train MAE: 0.3525, Val MAE: 0.4653
Epoch 7/1000, Train MAE: 0.3404, Val MAE: 0.4684
Epoch 8/1000, Train MAE: 0.3376, Val MAE: 0.4504
Epoch 9/1000, Train MAE: 0.3370, Val MAE: 0.4442
Epoch 10/1000, Train MAE: 0.3432, Val MAE: 0.4671
Epoch 11/1000, Train MAE: 0.3526, Val MAE: 0.4601
Epoch 12/1000, Train MAE: 0.3528, Val MAE: 0.4487
Epoch 13/1000, Train MAE: 0.3531, Val MAE: 0.4590
Epoch 14/1000, Train MAE: 0.3607, Val MAE: 0.4712
Epoch 15/1000, Train MAE: 0.3443, Val MAE: 0.4643
Epoch 16/1000, Train MAE: 0.3436, Val MAE: 0.5205
Epoch 17/1000, Train MAE: 0.3738, Val MAE: 0.4509
Epoch 18/1000, Train MAE: 0.3489, Val MAE: 0.4610
Epoch 19/1000, Train MAE: 0.3410, Val MAE: 0.4714
Epoch 20/1000, Train MAE: 0.3458, Val MAE: 0.4467
Epoch 21/

In [ ]:
torch.save(model.state_dict(), 'pytorch_exercise/AI_Phys_Hw_2/model_demo2.pth')